In [ ]:
pip install numpy matplotlib ucimlrepo scikit-learn --quiet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
import itertools

##############################################################################
# 1) Carrega o dataset manualmente via CSV (sem usar ucimlrepo)
##############################################################################
def load_dataset_once(csv_path: str = "ILPD.csv"):
    col_names = ["Age", "Gender", "Total_Bilirubin", "Direct_Bilirubin", 
                 "Alkaline_Phosphotase", "Alamine_Aminotransferase", 
                 "Aspartate_Aminotransferase", "Total_Proteins", 
                 "Albumin", "A/G_Ratio", "Selector"]
    df = pd.read_csv(csv_path, header=None, names=col_names)
    # Remoção de linhas com valores faltantes
    df.dropna(inplace=True)
    
    # Separe features e target
    X = df.drop(columns=["Selector"])
    y = df["Selector"].values
    
    # Converte a coluna Gender para numérico, se existir
    if "Gender" in X.columns:
        X["Gender"] = X["Gender"].map({"Female": 0, "Male": 1})
    
    # Reindexa os rótulos para começar em 0 (se os rótulos originais forem 1 e 2)
    y = y - 1

    # Normaliza as features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y

##############################################################################
# Funções auxiliares (one_hot_encode, train_model, evaluate_model) iguais
##############################################################################
def one_hot_encode(y: np.ndarray, num_classes: int) -> np.ndarray:
    """Converte um vetor de rótulos em codificação one-hot."""
    return np.eye(num_classes)[y]

def train_model(X_train: np.ndarray, y_train: np.ndarray, run: int,
                hidden_layers: tuple, learning_rate: float, momentum: float,
                max_epochs: int, convergence_threshold: float, patience: int,
                activation: str, num_classes: int):
    """
    Treina o MLPClassifier de forma incremental utilizando early stopping.
    """
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layers,
                        activation=activation,
                        solver='sgd',
                        learning_rate_init=learning_rate,
                        momentum=momentum,
                        max_iter=1,  # Treina 1 época por chamada
                        warm_start=True,  # permite treinamento incremental
                        shuffle=True,
                        random_state=run,
                        verbose=False)

    loss_history = []
    start_time = time.time()
    epochs_without_improvement = 0
    prev_loss = np.inf
    epoch = 0

    while epoch < max_epochs:
        if epoch == 0:
            mlp.partial_fit(X_train, y_train, classes=np.unique(y_train))
        else:
            mlp.partial_fit(X_train, y_train)

        current_loss = mlp.loss_
        loss_history.append(current_loss)
        epoch += 1

        # Verifica melhoria do loss
        improvement = abs(prev_loss - current_loss)
        if improvement < convergence_threshold:
            epochs_without_improvement += 1
        else:
            epochs_without_improvement = 0
        prev_loss = current_loss

        # Early stopping
        if epochs_without_improvement >= patience:
            break

    training_time = time.time() - start_time
    return mlp, loss_history, epoch, training_time

def evaluate_model(model: MLPClassifier, X_train: np.ndarray, y_train: np.ndarray,
                   X_test: np.ndarray, y_test: np.ndarray, num_classes: int):
    """
    Avalia o modelo treinado utilizando diversas métricas.
    """
    y_pred = model.predict(X_test)
    y_train_proba = model.predict_proba(X_train)
    y_test_proba = model.predict_proba(X_test)

    y_train_one_hot = one_hot_encode(y_train, num_classes)
    y_test_one_hot = one_hot_encode(y_test, num_classes)

    train_mse = mean_squared_error(y_train_one_hot, y_train_proba)
    test_mse = mean_squared_error(y_test_one_hot, y_test_proba)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

    return acc, prec, rec, f1, train_mse, test_mse

def run_experiment_config(n_runs: int, config: dict, X: np.ndarray, y: np.ndarray):
    """
    Executa n_runs do experimento com a configuração definida em config
    e retorna a acurácia média.
    """
    num_classes = len(np.unique(y))
    accuracies = []

    for run in range(n_runs):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, stratify=y, random_state=run
        )

        model, loss_history, epochs, training_time = train_model(
            X_train, y_train, run,
            hidden_layers=config['hidden_layers'],
            learning_rate=config['learning_rate'],
            momentum=config['momentum'],
            max_epochs=config['max_epochs'],
            convergence_threshold=config['convergence_threshold'],
            patience=config['patience'],
            activation=config['activation'],
            num_classes=num_classes
        )

        acc, prec, rec, f1, train_mse, test_mse = evaluate_model(
            model, X_train, y_train, X_test, y_test, num_classes
        )
        accuracies.append(acc)

    avg_accuracy = np.mean(accuracies)
    return avg_accuracy

def grid_search():
    """
    Realiza grid search exaustivo sobre as configurações definidas e 
    pula as primeiras 366 combinações, retomando a partir da 367.
    """
    # Carrega os dados localmente
    X_GLOBAL, Y_GLOBAL = load_dataset_once("ILPD.csv")

    # Parâmetros do grid
    grid_params = {
        'momentum': [0.0, 0.1, 0.2, 0.3],
        'learning_rate': [0.001, 0.01, 0.1, 0.3],
        'hidden_neurons': [10, 20, 40, 60, 80],
        'hidden_layers_count': [1, 2, 3],
        'activation': ['logistic', 'relu'],
        'patience': [10, 15],
        'convergence_threshold': [1e-4, 1e-5, 1e-6]
    }

    n_runs = 10
    max_epochs = 10000
    threshold = 0.75

    total_configs = np.prod([len(v) for v in grid_params.values()])

    # Ajuste aqui o número de configurações já testadas
    already_tested = 0

    config_counter = 0

    for (momentum, learning_rate, hidden_neurons, hidden_layers_count,
         activation, patience, convergence_threshold) in itertools.product(
            grid_params['momentum'],
            grid_params['learning_rate'],
            grid_params['hidden_neurons'],
            grid_params['hidden_layers_count'],
            grid_params['activation'],
            grid_params['patience'],
            grid_params['convergence_threshold']):

        config_counter += 1
        if config_counter <= already_tested:
            continue  # Pula as já executadas

        config = {
            'momentum': momentum,
            'learning_rate': learning_rate,
            'hidden_layers': (hidden_neurons,) * hidden_layers_count,
            'activation': activation,
            'patience': patience,
            'convergence_threshold': convergence_threshold,
            'max_epochs': max_epochs
        }

        print(f"\nTestando configuração {config_counter}/{total_configs}: {config}")
        avg_acc = run_experiment_config(n_runs, config, X_GLOBAL, Y_GLOBAL)
        print(f"  -> Acurácia média: {avg_acc:.4f}")

        if avg_acc >= threshold:
            print(f"\n*** CONFIGURAÇÃO PROMISSORA ACHADA ***")
            print(f"Config: {config}")
            print(f"Acurácia média: {avg_acc:.4f}\n")

if __name__ == "__main__":
    grid_search()



Testando configuração 1/2880: {'momentum': 0.0, 'learning_rate': 0.001, 'hidden_layers': (10,), 'activation': 'logistic', 'patience': 10, 'convergence_threshold': 0.0001, 'max_epochs': 10000}


C:\Users\João Lucas\AppData\Roaming\Python\Python313\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


  -> Acurácia média: 0.7098

Testando configuração 2/2880: {'momentum': 0.0, 'learning_rate': 0.001, 'hidden_layers': (10,), 'activation': 'logistic', 'patience': 10, 'convergence_threshold': 1e-05, 'max_epochs': 10000}
